<a href="https://colab.research.google.com/github/grackner/dsc_261_final_project/blob/main/phi_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Phi Text Generation

url: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct?library=transformers

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import uuid
import pandas as pd
import kagglehub
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True, dtype='float16')

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_fea

In [4]:
# Test inference
messages =[
    {"role": "user", "content": "Who is Michael Jordan?"},
]
inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=100, use_cache=False)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Michael Jordan, born February 17, 1963, in Brooklyn, New York, is a retired professional basketball player widely regarded as one of the greatest basketball players of all time. He played 15 seasons in the NBA, primarily with the Chicago Bulls. Jordan won six NBA championships and earned five Most Valuable Player (MVP) Awards. He is also a five-time NBA All-Star and a 14-time NBA All-Star


In [6]:
# Create test dataset
phi_df = pd.DataFrame(columns=['uuid', 'topic', 'generated_article'])

In [7]:
topics = ['US - Crime + Justice', 'World - Africa', 'World - Americas', 'World - Asia', 'World - Australia', 'World - China', 'World - Europe', 'World - India', 'World - Middle East', 'World - United Kingdom',
          'World - India', 'World - Middle East', 'World - United Kingdom', 'Politics - CNN Polls', 'Politics - Elections', 'Business - Tech', 'Business - Media', 'Business - Markets',
          'Business - Pre-markets', 'Business - After-Hours', 'Business - Investing', 'Business - Markets Now', 'Health - Fitness', 'Health - Food', 'Health - Sleep', 'Health - Mindfulness',
          'Health - Relationships', 'Entertainment - Movies', 'Entertainment - Television', 'Entertainment - Celebrity', 'Tech - Innovate', 'Tech - Foreseeable Future', 'Tech - Innovative Cities',
          'Style - Arts', 'Style - Design', 'Style - Fashion', 'Style - Architecture', 'Style - Luxury', 'Style - Beauty', 'Travel - Destinations', 'Travel - Food & Drink', 'Travel - Lodging and Hotels',
          'Travel - News', 'Sports - Pro Football', 'Sports - College Football', 'Sports - Basketball', 'Sports - Baseball', 'Sports - Soccer', 'Sports - Olympics', 'Sports - Hockey',
          'Science - Space', 'Science - Life', 'Science - Medicine', 'Science - Climate', 'Science - Solutions', 'Science - Weather']

In [8]:
def generate_article(topic):
  messages = [
    {
        "role": "user",
        "content": f"""Write a full news article in the style of CNN or DailyMail.
                    The story should sound realistic, factual, and human-written.
                    Use natural journalistic language with short and medium-length sentences.
                    Start with a strong lead paragraph summarizing who, what, where, and when.
                    Then expand with quotes, context, background, and a final paragraph about next steps or reactions.
                    Include realistic numbers, dates, and locations.
                    The article should be about {topic}.
                    Add 1–3 short quotes attributed to plausible people (officials, witnesses, or experts).
                    Use neutral tone — no opinions, exaggeration, or bullet points.
                    Output only the article text (no headline, no lists, no explanation, no “to summarize”).
                    End cleanly after several paragraphs.
                  """
    },
  ]
  inputs = tokenizer.apply_chat_template(
          messages,
          add_generation_prompt=True,
          tokenize=True,
          return_dict=True,
          return_tensors="pt",
  ).to(model.device)

  outputs = model.generate(**inputs, max_new_tokens=750, use_cache=False, do_sample=True, temperature=0.9, top_p=0.95,top_k=50)
  response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
  return response

In [9]:
# Generate articles
rng = np.random.default_rng()
n = 4 # Number of datapoints to create
for i in range(0, n + 1):
  print(i)
  # Get random num between 0 and length of list
  random_integer = rng.integers(low=0, high=len(topics))
  topic = topics[random_integer]
  # Add unique identifier for the row
  phi_df.loc[i, 'uuid'] = str(uuid.uuid4())
  phi_df.loc[i, 'topic'] = topic
  response = generate_article(topic)
  phi_df.loc[i, 'generated_article'] = response

0
1
2
3
4


In [11]:
phi_df.head()

,uuid,topic,generated_article
0,c35bd602-dfc1-4c6f-a44b-517e28e6d0f0,Sports - Baseball,The New York Yankees have announced they will ...
1,fa4dc61b-44f2-4be3-a837-ad768e3c0c7c,Tech - Innovate,"On March 15th, 2023, Innovate Corp, a leading ..."
2,51e7494c-3a24-4858-8068-5710a8588d52,Entertainment - Television,"The newest reality TV show, ""Culinary Conquest..."
3,66232914-309c-486e-8997-df42bdcb811b,Travel - Lodging and Hotels,"On Tuesday, the Grandiose Hotel in downtown La..."
4,cf455341-ec8c-4150-9ba9-f8217080c9b6,Sports - Olympics,The Tokyo Olympic Games continue to dazzle wit...


In [12]:
phi_df.loc[3, 'generated_article']

'On Tuesday, the Grandiose Hotel in downtown Las Vegas became the center of an unprecedented event as guests reported an unexpected influx of marine wildlife into the hotel\'s luxurious pool area. The incident occurred early in the morning when hotel guests woke up to find several dolphins and a young sea turtle swimming in the hotel pool.\n\nThe Grandiose Hotel\'s management immediately took action to ensure the safety of both the guests and the marine creatures. "We want to emphasize that the hotel was never responsible for this situation, and we have never intentionally housed marine life within our pool area," said John Smith, the hotel\'s CEO. "We are now working closely with local wildlife authorities to ensure the well-being of the animals and to take necessary actions."\n\nAccording to witnesses, the event began when a group of tourists spotted a sea turtle struggling to swim near the edge of the pool. Concerned for its safety, they reached out to hotel staff, who quickly spran

In [ ]:
phi_df.to_csv("phi_outputs.csv")

In [ ]:
## Load dataset
# path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

# print("Path to dataset files:", path)

# train_df = pd.read_csv(path + "/cnn_dailymail/train.csv")
# test_df = pd.read_csv(path + "/cnn_dailymail/test.csv")
# val_df = pd.read_csv(path + "/cnn_dailymail/validation.csv")

In [ ]:
## Loop through the train dataset
# for index, row in train_df.head(1).iterrows():
#   print(index)
#   # Get article
#   article = row["article"]
#   # Cut down to 100 words
#   article = " ".join(article.split()[:100])
#   # Format the query
#   messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "text", "text": f"Write a news article do not include a title"}
#             # {"type": "text", "text": f"Write an article with a similar style to the following article example from CNN: {article}"}
#         ]
#     },
#   ]
#   # Store query in df # TODO: Should happen in data cleaning?
#   print(article)
#   train_df.loc[index, 'query_article'] = messages[0]['content'][0]['text']
#   ## Run query through inference
#   inputs = processor.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt",
#   ).to(model.device)

#   outputs = model.generate(**inputs, max_new_tokens=100)
#   response = processor.decode(outputs[0][inputs["input_ids"].shape[-1]:])
#   # Store answer in df
#   train_df.loc[index, 'model_output'] = response